### Random Forest Classifier

In [28]:
import numpy as np
np.random.seed(456)
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

data = iris['data']
out = iris['target']
train_x, test_x, train_y, test_y = train_test_split(data, out, test_size=0.33)

In [29]:
sklearn_model = RandomForestClassifier(class_weight="balanced", n_estimators=20)
sklearn_model.fit(train_x, train_y)

train_y_pred = sklearn_model.predict(train_x)
test_y_pred = sklearn_model.predict(test_x)

weighted_score = accuracy_score(train_y, train_y_pred)
print("Weighted train Classification Accuracy: %f" % weighted_score)
weighted_score = accuracy_score(test_y, test_y_pred)
print("Weighted test Classification Accuracy: %f" % weighted_score)

Weighted train Classification Accuracy: 1.000000
Weighted test Classification Accuracy: 0.980000


### Model Evaluator

In [37]:
import tensorflow as tf

def eval_tox21_hyperparams(n_hidden=50, n_layers=1, learning_rate=.001,
                           dropout_prob=0.5, n_epochs=45, batch_size=100,
                           weight_positives=True):

    print("---------------------------------------------")
    print("Model hyperparameters")
    print("n_hidden = %d" % n_hidden)
    print("n_layers = %d" % n_layers)
    print("learning_rate = %f" % learning_rate)
    print("n_epochs = %d" % n_epochs)
    print("batch_size = %d" % batch_size)
    print("weight_positives = %s" % str(weight_positives))
    print("dropout_prob = %f" % dropout_prob)
    print("---------------------------------------------")
    
    d = 4
    x = tf.placeholder(tf.float32, (None, d))
    y = tf.placeholder(tf.float32, (None))

    w = tf.Variable(tf.random_normal((d, n_hidden)))
    b = tf.Variable(tf.random_normal((n_hidden,)))
    x_hidden = tf.nn.relu(tf.matmul(x, w) + b)
    keep_prob = tf.placeholder(tf.float32)
    x_hidden = tf.nn.dropout(x_hidden, keep_prob)

    w_out = tf.Variable(tf.random_normal((n_hidden, 1)))
    b_out = tf.Variable(tf.random_normal((1,)))
    y_logit = tf.matmul(x_hidden, w_out) + b_out
    y_pred = tf.round(tf.sigmoid(y_logit))

    y_expand = tf.expand_dims(y, 1)
    entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y_expand)
    l = tf.reduce_sum(entropy)

    train_op = tf.train.AdamOptimizer(learning_rate).minimize(l)
    
    N = train_x.shape[0]
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        step = 0
        for epoch in range(n_epochs):
            pos = 0
            while pos < N:
                feed_dict = {x: train_x[pos:pos+batch_size], y: train_y[pos:pos+batch_size], keep_prob: dropout_prob}
                _, loss = sess.run([train_op, l], feed_dict=feed_dict)
                step += 1
                pos += batch_size
        y_pred_out = sess.run(y_pred, feed_dict={x: test_x, keep_prob: 1.0})
        print("Accuracy Score: {}".format(accuracy_score(test_y, y_pred_out)))


### Grid Search

In [38]:
scores = {}
n_reps = 3
hidden_sizes = [50]
epochs = [10]
dropouts = [.5, 1.0]
num_layers = [1, 2]

for rep in range(n_reps):
    for n_epochs in epochs:
        for hidden_size in hidden_sizes:
            for dropout in dropouts:
                for n_layers in num_layers:
                    eval_tox21_hyperparams(n_hidden=hidden_size, n_epochs=n_epochs, dropout_prob=dropout, n_layers=n_layers)

---------------------------------------------
Model hyperparameters
n_hidden = 50
n_layers = 1
learning_rate = 0.001000
n_epochs = 10
batch_size = 100
weight_positives = True
dropout_prob = 0.500000
---------------------------------------------


ValueError: Dimensions must be equal, but are 50 and 4 for 'MatMul_4' (op: 'MatMul') with input shapes: [?,50], [4,1].